In [3]:
# Importar configuración
import sys
sys.path.append("/scripts/config")
from db_config import db_configMaria

print("Contenido de db_configMB:")
for key, value in db_configMaria.items():
    print(f"{key} = {value}")

Contenido de db_configMB:
server = mariadb
port = 3306
database = etl_config
user = root
password = StrongPass123


In [11]:
import sys
sys.path.append("/scripts/config")   # para encontrar db_config
sys.path.append("/scripts")          # para encontrar tu archivo parametrossh.py

from parametrossh import get_params

# Probar con SPARK
spark_params = get_params("SPARK", 1)
print("✅ Parámetros SPARK:", spark_params)




✅ Parámetros SPARK: {'spark.master': 'spark://spark-master:7077', 'spark.executor.instances': '1', 'spark.executor.cores': '1', 'spark.executor.memory': '2g', 'spark.driver.memory': '1g', 'spark.jars': '/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar', 'spark.driver.extraClassPath': '/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar', 'spark.executor.extraClassPath': '/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar'}


In [23]:
# Probar con SQLSERVER
sqlserver_params = get_params("SQLSERVER", 1)
print("✅ Parámetros SQLSERVER:", sqlserver_params)

✅ Parámetros SQLSERVER: {'sqlserver.host': 'host.docker.internal', 'sqlserver.port': '1433', 'sqlserver.database': 'olva', 'sqlserver.user': 'etl_user', 'sqlserver.password': 'StrongPass123', 'sqlserver.driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver', 'sqlserver.url': 'jdbc:sqlserver://host.docker.internal:1433;databaseName=olva;encrypt=false;trustServerCertificate=true'}


In [13]:
from pyspark.sql import SparkSession
from parametrossh import get_params   # ya la tienes lista

def init_spark(ambiente=1, app_name="ProyectoHive"):
    # 1. Recuperar parámetros desde MariaDB
    spark_params = get_params("SPARK", ambiente)

    # 2. Construir el SparkSession con esos parámetros
    builder = SparkSession.builder.appName(app_name)
    for key, value in spark_params.items():
        builder = builder.config(key, value)

    # 3. Crear sesión
    spark = builder.getOrCreate()
    return spark


In [17]:
import importlib, parametrossh
importlib.reload(parametrossh)

from parametrossh import init_spark


In [16]:
import parametrossh
print(dir(parametrossh))


['SparkSession', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'db_configMaria', 'get_params', 'init_spark', 'mysql', 'pymysql']


In [18]:
import sys
sys.path.append("/scripts")

from parametrossh import init_spark

spark = init_spark(1, app_name="PruebaParametros")
print("✅ SparkSession levantada correctamente")
print("Versión:", spark.version)
print("Master URL:", spark.sparkContext.master)


25/09/22 20:25:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ SparkSession levantada correctamente
Versión: 2.4.5
Master URL: spark://spark-master:7077


In [25]:
import sys
sys.path.append("/scripts")

import importlib, parametrossh
importlib.reload(parametrossh)

hdfs_paths = parametrossh.get_hdfs_paths(1)
print("✅ Rutas HDFS:", hdfs_paths)


✅ Rutas HDFS: {'hdfs.bronze': 'hdfs://namenode:8020/bronze', 'hdfs.silver': 'hdfs://namenode:8020/silver', 'hdfs.data': 'hdfs://namenode:8020/data', 'hdfs.tmp': 'hdfs://namenode:8020/tmp', 'hdfs.user': 'hdfs://namenode:8020/user'}


In [26]:
import sys
sys.path.append("/scripts")

import importlib, parametrossh
importlib.reload(parametrossh)

# ahora sí
hdfs_params = parametrossh.get_params_full("HDFS", 1)

for row in hdfs_params:
    print(f"ORDEN={row['ORDEN']} | {row['PARAMETRO']} = {row['VALOR']}")


ORDEN=1 | hdfs.bronze = hdfs://namenode:8020/bronze
ORDEN=2 | hdfs.silver = hdfs://namenode:8020/silver
ORDEN=3 | hdfs.data = hdfs://namenode:8020/data
ORDEN=4 | hdfs.tmp = hdfs://namenode:8020/tmp
ORDEN=5 | hdfs.user = hdfs://namenode:8020/user


In [29]:
bronze_path = get_params("HDFS", 1)["hdfs.bronze"]

print(bronze_path)


hdfs://namenode:8020/bronze


In [33]:
# Ya tienes la ruta
bronze_path = get_params("HDFS", 1)["hdfs.bronze"]
print(f"La ruta bronze: {bronze_path}")

# Ahora vemos qué hay dentro - Configuración corregida para HDFS
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hdfs_uri = spark.sparkContext._jvm.java.net.URI.create("hdfs://namenode:8020")
fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(hdfs_uri, hadoop_conf)
path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path(bronze_path)

print(f"\nContenido de bronze:")
print("="*50)

file_status = fs.listStatus(path)
for status in file_status:
    file_path = status.getPath().toString()
    is_dir = status.isDirectory()
    name = file_path.split('/')[-1]
    
    if is_dir:
        print(f"📁 {name}")
    else:
        size = status.getLen()
        print(f"📄 {name} ({size} bytes)")

La ruta bronze: hdfs://namenode:8020/bronze

Contenido de bronze:
📁 cliente_df
📁 clientes
📁 clientes_partitioned
📁 clientes_watermark
📁 data_df
📁 df_pedidos
📁 df_pedidos_hive
📁 practica
📁 update_clientes_watermark
📁 validacion_df


In [35]:
# Ya tienes la ruta base de bronze
bronze_path = get_params("HDFS", 1)["hdfs.bronze"]

# Elegir una tabla para trabajar (cambia por la que necesites)
tabla_elegida = "df_pedidos"  # o "cliente_df", "clientes", etc.

# Construir la ruta completa
ruta_tabla = f"{bronze_path}/{tabla_elegida}"
print(f"Trabajando con: {ruta_tabla}")

# Leer la tabla
df = spark.read.parquet(ruta_tabla)

Trabajando con: hdfs://namenode:8020/bronze/df_pedidos


In [36]:
df.show(10)

[Stage 1:>                                                          (0 + 1) / 1]

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
|       6|        1| 80.00| 2025-09-06|2025-09-14 19:36:...|      null|
|       7|        2| 60.00| 2025-09-07|2025-09-14 19:36:...|      null|
|       8|        3| 90.00| 2025-09-08|2025-09-14 19:36:...|      null|
|       9|        4|110.00| 2025-09-09|2025-09-14 19:36:...|      null|
|      10|        5| 75.00| 2025-09-10|2025-09-14 19:36:...|      null|
+--------+---------+------+-----------+--------------------+----